### 데이터 값과 기업정보 최종 결과
* 결산년도 2017년 ~ 2021년
* 상장법인(유가증권+코스닥)

* 데이터 값 목록 가져오기

In [19]:
import pandas as pd

# 데이터값 파일
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/items_from_section/items_all.csv"
df_items_all = pd.read_csv(file_path, dtype=object)

# 기업개황정보 파일 읽기
file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/company/corp_info_all.csv"
df_corp_info_all = pd.read_csv(file_path, dtype=object)

In [20]:
print(df_items_all.shape[0])
df_items_all.head(3)

2444


,corp_code,stock_code,corp_cls,corp_name,rpt_num,rpt_name,doc_title,doc_result,section_title,unit,val_a,val_b,item_result,doc_url
0,00100939,000860,Y,강남제비스코,20181231000114,[기재정정]사업보고서 (2013.12),NaN,No Financial Statement Notes,NaN,NaN,NaN,NaN,NaN,NaN
1,00100939,000860,Y,강남제비스코,20181231000137,[기재정정]사업보고서 (2017.12),5. 재무제표 주석,OK,21. 확정급여제도 :,백만원,23399.0,25631.0,"기금이 적립된 확정급여채무의 현재가치, 사외적립자산의 공정가치(*)",http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
2,00100939,000860,Y,강남제비스코,20181231000103,[기재정정]사업보고서 (2011.12),NaN,No Financial Statement Notes,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# df_items_all[df_items_all['corp_name'].str.contains('송원산업')]

* 데이터값과 기업개황정보 결합

In [21]:
df_result = pd.merge(df_items_all, df_corp_info_all[['corp_code', 'jurir_no', 'bizr_no', 'induty_code']], 
                     how='left', left_on='corp_code', right_on='corp_code')

In [22]:
# 금액을 백만원 단위로 변환
def change_val(v, unit):
    if unit == '백만원': amt = float(v)
    elif unit == '천원': amt = round(float(v) / 1000, 0)
    elif unit == '원': amt = round(float(v) / 1000000, 0)
    else: amt = 0
    return amt

In [23]:
# 금액을 백만원 단위로 변환한 항목을 추가
df_result['m_val_a'] = df_result.apply(lambda x : change_val(x['val_a'], x['unit']), axis=1)
df_result['m_val_b'] = df_result.apply(lambda x : change_val(x['val_b'], x['unit']), axis=1)

In [24]:
def year_month(str):
    substr = str[-8:-1]
    y, m = substr.split(".")
    return y, m

df_result['year'], df_result['month'] = zip(*df_result['rpt_name'].apply(year_month))

In [32]:
"""
# Section Title은 찾았지만 금액 찾기(Value Table)는 실패한 경우: item_result == "No Contents"
# 금액 찾기 오류 발생 건: item_result == "Exception"
# 금액 찾기 오류 발생 건: val_a == "-1" or val_b == "-1"
"""
def check_needed(str):
    # str[0]=item_result, str[1]=val_a, str[2]=val_b
    if str[0] == "No Contents" or str[0] == "Exception" or str[1] == "-1" or str[2] == "-1":
        check_comment = "Manual Check Needed"
    else:
        check_comment = None
    return check_comment

df_result['check'] = df_result[['item_result', 'val_a', 'val_b']].apply(check_needed, axis=1)
# df_result["check"] = ["Manual Check Needed" if s == "No Contents" or s == "Exception" else None for s in df_result["item_result"]]

In [38]:
# 컬럼순서 재배치
df_result = df_result[['induty_code', 'jurir_no', 'bizr_no', 'corp_code', 'stock_code', \
                       'corp_cls', 'corp_name', 'rpt_num', 'year', 'month', 'rpt_name', 'doc_title', 'doc_result', \
                       'section_title', 'unit', 'val_a', 'val_b', 'm_val_a', 'm_val_b', \
                       'item_result', 'check', 'doc_url']]
# 정렬
df_result.sort_values(by=['induty_code', 'jurir_no', 'corp_code', 'rpt_num'], inplace=True)
df_result.head(5)

,induty_code,jurir_no,bizr_no,corp_code,stock_code,corp_cls,corp_name,rpt_num,year,month,...,doc_result,section_title,unit,val_a,val_b,m_val_a,m_val_b,item_result,check,doc_url
1109,00102,1101110104820,1048127881,00141875,006090,Y,사조오양,20170331003000,2016,12,...,OK,18. 종업원급여,천원,10802905.0,6213761.0,10803.0,6214.0,"확정급여채무의 현재가치, 사외적립자산의 공정가치",None,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1111,00102,1101110104820,1048127881,00141875,006090,Y,사조오양,20180402003435,2017,12,...,OK,18. 종업원급여,천원,12241819.0,7931515.0,12242.0,7932.0,"확정급여채무의 현재가치, 사외적립자산의 공정가치",None,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1110,00102,1101110104820,1048127881,00141875,006090,Y,사조오양,20190401003846,2018,12,...,OK,18. 종업원급여,천원,13764813.0,8456699.0,13765.0,8457.0,"확정급여채무의 현재가치, 사외적립자산의 공정가치",None,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1112,00102,1101110104820,1048127881,00141875,006090,Y,사조오양,20200330002828,2019,12,...,OK,18. 종업원급여,천원,16436592.0,9870436.0,16437.0,9870.0,"확정급여채무의 현재가치, 사외적립자산의 공정가치",None,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1108,00102,1101110104820,1048127881,00141875,006090,Y,사조오양,20210318001029,2020,12,...,OK,18. 종업원급여,천원,17390099.0,10501739.0,17390.0,10502.0,"확정급여채무의 현재가치, 사외적립자산의 공정가치",None,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...


In [52]:
df_result_mfg = df_result[df_result['induty_code'].str[0:2].between('10', '34', inclusive='both')]
df_result_mfg = df_result_mfg.loc[(df_result_mfg.corp_cls == 'Y') & (df_result_mfg.rpt_name.str[-8:-4] >= '2017') & (df_result_mfg.rpt_name.str[-8:-4] <= '2021'), ]
df_result_mfg.shape[0]

1354

In [53]:
corp_codes = df_result_mfg['corp_code'].unique()
len(corp_codes)

274

In [54]:
# default max_rows=60 & max_columns=20
pd.options.display.max_rows = 60
pd.options.display.max_columns = 20
pd.set_option('display.max_rows', None) ## 모든 행을 출력한다. 
df_result_mfg[df_result_mfg.check == "Manual Check Needed"]

,induty_code,jurir_no,bizr_no,corp_code,stock_code,corp_cls,corp_name,rpt_num,year,month,...,doc_result,section_title,unit,val_a,val_b,m_val_a,m_val_b,item_result,check,doc_url
569,11122,1901110002721,6088107131,00121543,033920,Y,무학,20190225002450,2017,12,...,OK,17. 퇴직급여제도,NaN,0.0,0.0,0.0,0.0,No Contents,Manual Check Needed,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
566,11122,1901110002721,6088107131,00121543,033920,Y,무학,20190401004891,2018,12,...,OK,17. 퇴직급여제도,NaN,0.0,0.0,0.0,0.0,No Contents,Manual Check Needed,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
563,11122,1901110002721,6088107131,00121543,033920,Y,무학,20200330004351,2019,12,...,OK,18. 퇴직급여제도,NaN,0.0,0.0,0.0,0.0,No Contents,Manual Check Needed,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
565,11122,1901110002721,6088107131,00121543,033920,Y,무학,20210319001404,2020,12,...,OK,18. 퇴직급여제도,NaN,0.0,0.0,0.0,0.0,No Contents,Manual Check Needed,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
567,11122,1901110002721,6088107131,00121543,033920,Y,무학,20220318000779,2021,12,...,OK,18. 퇴직급여제도,NaN,0.0,0.0,0.0,0.0,No Contents,Manual Check Needed,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1226,13101,1101110041717,1168140780,00146269,003200,Y,일신방직,20180330000861,2017,12,...,OK,21. 퇴직급여채무,NaN,0.0,0.0,0.0,0.0,No Contents,Manual Check Needed,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1227,13101,1101110041717,1168140780,00146269,003200,Y,일신방직,20190401001048,2018,12,...,OK,20. 퇴직급여제도,NaN,0.0,0.0,0.0,0.0,No Contents,Manual Check Needed,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1228,13101,1101110041717,1168140780,00146269,003200,Y,일신방직,20200330002341,2019,12,...,OK,22. 퇴직급여,NaN,0.0,0.0,0.0,0.0,No Contents,Manual Check Needed,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1229,13101,1101110041717,1168140780,00146269,003200,Y,일신방직,20210317000634,2020,12,...,OK,20. 퇴직급여,NaN,0.0,0.0,0.0,0.0,No Contents,Manual Check Needed,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...
1230,13101,1101110041717,1168140780,00146269,003200,Y,일신방직,20220725000075,2021,12,...,OK,20. 퇴직급여,NaN,0.0,0.0,0.0,0.0,No Contents,Manual Check Needed,http://dart.fss.or.kr/report/viewer.do?rcpNo=2...


In [55]:
import sys
module_path = "D:\PythonProject\data-gatherer\dart_fs_notes"
sys.path.append(module_path)
import myutil

file_path = "D:/PythonProject/data-gatherer/dart_fs_notes/output/"
file_name = 'result_mfg_' + myutil.now_dt_str() + '.csv'
df_result_mfg.to_csv(file_path + file_name, index=False)